In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
ed = tfp.edward2
import numpy as np
from tfpmodels import *

from future_features import tape, SoftmaxCentered

/home/folzd/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Mapper:
    _positive_distributions = [tfd.InverseGamma, tfd.Gamma]
    _simplex_distributions = [tfd.Dirichlet]
    _tril_distributions = [tfd.Wishart]

    def __init__(self, model, model_name, observed_variable_names, *args, **kwargs):
        self.model = model
        self.model_name = model_name
        self.observed_variable_names = observed_variable_names
        self.log_joint_fn = ed.make_log_joint_fn(model) 
        self._args = args
        self._kwargs = kwargs
        with tape() as self.tape:
            self.output = self.model(*args, **kwargs)
        with tf.variable_scope(self.model_name, reuse=tf.AUTO_REUSE):
            self.variable_names = [key for key in self.tape.keys() if key not in self.observed_variable_names]
            self.variable_dist = {key: self.tape[key].distribution for key in self.variable_names}
            self.variable_shapes = {key: self.tape[key].shape for key in self.variable_names}
            self.transforms = {key: self.get_bijector(self.tape[key]) for key in self.variable_names}
            self.unconstrained_variable_shapes = {key: self.transforms[key].inverse_event_shape(val) for key, val in self.variable_shapes.items()}
            self.unconstrained_variables = {key: tf.get_variable(key, shape=self.unconstrained_variable_shapes[key]) for key in self.variable_names}
            self.variables = {key: self.transforms[key].forward(val) for key, val in self.unconstrained_variables.items()}
        
    def get_bijector(self, random_variable):
        distribution = random_variable.distribution
        if distribution.__class__ in self._positive_distributions:
            return tfb.Softplus() #tfp.trainable_distributions.softplus_and_shift(variable)
        elif distribution.__class__ in self._simplex_distributions:
            return SoftmaxCentered()
        elif distribution.__class__ in self._tril_distributions:
            return tfb.ScaleTriL()
        else:
            return tfb.Identity()

    def map_neg_log_joint_fn(self, **kwargs):
        return -self.log_joint_fn(*self._args, **self._kwargs, **self.variables, **kwargs)

    def map_optimizer(self, **kwargs):
        map_neg_log_joint = self.map_neg_log_joint_fn(**kwargs)
        return map_neg_log_joint, tf.contrib.opt.ScipyOptimizerInterface(map_neg_log_joint, self.unconstrained_variables.values())

    def assigner(self, **kwargs):
        assign_ops = []
        for key, val in kwargs.items():
            assign_ops.append(tf.assign(self.unconstrained_variables[key], self.transforms[key].inverse(val)))
        return tf.group(assign_ops)

In [3]:
N = 100
n_components = 4
n_features = 2
Mapper(mixtureOfGaussians, 'mog', observed_variable_names=['data'], n_observations=N, n_components=n_components, n_features=n_features)

In [4]:
class VIper:

    _positive_distributions = [tfd.InverseGamma, tfd.Gamma]
    _simplex_distributions = [tfd.Dirichlet]
    _tril_distributions = [tfd.Wishart]
    p_to_q_map = {
        
    }

    def __init__(self, model, model_name, observed_variable_names, *args, **kwargs):
        self.model = model
        self.model_name = model_name
        self.observed_variable_names = observed_variable_names
        self.log_joint_fn = ed.make_log_joint_fn(model) 
        self._args = args
        self._kwargs = kwargs
        with tape() as self.tape:
            self.output = self.model(*args, **kwargs)
        
    def variational_model(self, variational_parameters):
        with tf.variable_scope(self.model_name, reuse=tf.AUTO_REUSE):
            self.variable_names = [key for key in self.tape.keys() if key not in self.observed_variable_names]
            self.variable_shapes = {key: self.tape[key].shape for key in self.variable_names}
            self.variable_dist = {key: self.tape[key].distribution for key in self.variable_names}
            print(self.variable_dist)
            self.unconstrained_variables = {key: tf.get_variable(key, shape=self.variable_shapes[key]) for key in self.variable_names}
            self.variables = {key: self.wrap(val, self.variable_dist[key]) for key, val in self.unconstrained_variables.items()}        
        return variational_model
    
    def wrap(self, variable, distribution):
        if distribution.__class__ in self._positive_distributions:
            return tfp.trainable_distributions.softplus_and_shift(variable)
        if distribution.__class__ in self._simplex_distributions:
            return tf.nn.softmax(variable)
        if distribution.__class__ in self._tril_distributions:
            batch = variable.shape[0]
            dim = variable.shape[1]
            trildim = dim*(dim+1)//2
            return tfp.trainable_distributions.tril_with_diag_softplus_and_shift(tf.reshape(variable,(batch,-1))[:,:trildim])
        return variable

    def map_neg_log_joint_fn(self, **kwargs):
        return -self.log_joint_fn(*self._args, **self._kwargs, **self.variables, **kwargs)

    def map_optimizer(self, **kwargs):
        map_neg_log_joint = self.map_neg_log_joint_fn(**kwargs)
        return map_neg_log_joint, tf.contrib.opt.ScipyOptimizerInterface(map_neg_log_joint, self.unconstrained_variables.values())

    def assigner(self, **kwargs):
        assign_ops = []
        for key, val in kwargs.items():
            assign_ops.append(tf.assign(self.unconstrained_variables[key], val))
        return tf.group(assign_ops)

In [7]:
N = 100
n_components = 4
n_features = 2
adder = VIper(mixtureOfGaussians, 'mog', observed_variable_names=['data'], n_observations=N, n_components=n_components, n_features=n_features)

In [8]:
adder.variational_model(variational_parameters='lol')

{'mixture_weights': <tf.distributions.Dirichlet 'mixture_weights_2/' batch_shape=() event_shape=(4,) dtype=float32>, 'mixture_component_means': <tf.distributions.Normal 'mixture_component_means_2/' batch_shape=() event_shape=() dtype=float32>, 'mixture_component_covariances_cholesky': <tf.distributions.Wishart 'mixture_component_covariances_cholesky_2/' batch_shape=() event_shape=(2, 2) dtype=float32>}


ValueError: Trying to share variable mog/mixture_weights, but specified shape (4,) and found shape (3,).